# Week 5: Finishing the TTR Experiment

In this week, we put together everything we've learned so far this semester to tackle the full TTR experiment.

1. Removing puncutation with regular expressions

2. Iterating through files in a folder

3. Automatically determining sample size and producing standardized results

4. Writing CSV files

### 1. Removing Punctuation 

What we want a sophisticated "find and replace" operation. Let's start simple with `string.replace()` 

In [ ]:
sot4 = open("sign-of-four.txt", encoding="utf-8").read()
cocaine_exchange = sot4[1475:1843]
print(cocaine_exchange)

Let's say we would like to do three things to clean up this string:
* Replace the too-risqué word "cocaine" with "strawberry soda"
* Remove all punctuation
* Extract only the dialogue from this exchange, storing each piece of dialogue as an item in a list

In [ ]:
cocaine_exchange = cocaine_exchange.replace("cocaine", "stawberry soda")
print(cocaine_exchange)

In [ ]:
cocaine_exchange = cocaine_exchange.replace("?", "")
print(cocaine_exchange)

As you can see, `string.replace()` can remove punctuation... but we need to specify each piece of punctuation one-by-one, which is rather laborious.

In [ ]:
cocaine_exchange = cocaine_exchange.replace("“", "")
print(cocaine_exchange)

As for extracting all passages of dialogue, `string.replace()` offers nothing at all. It can only take particular strings and replace them with other strings.

This is where **regular expressions** come in...

## Python libraries

Python has a bunch of built-in functionality in libraries or modules that we need to import to be able to use.

The regular expression library is named "re"

In [ ]:
import re

### Using library functions

`re.sub()` takes three arguments: `re.sub("the string to replace", "the string to replace it with", the_variable_containing_the_text)`. 

Note that **the `re.sub()` function is NOT a mutating function** — so if you want to store its output, you need to explicitly stick it into a variable.

Here's how we would do our first two tasks:

In [ ]:
# reset our text
cocaine_exchange = sot4[1475:1843]

cocaine_exchange = re.sub("cocaine", "strawberry soda", cocaine_exchange)
print(cocaine_exchange)

In [ ]:
cocaine_exchange = re.sub("“", "", cocaine_exchange)
print(cocaine_exchange)

## Regular Expresssions

Regular Expressions are capable of doing so much more than this.

Let's explore some of what you can do with regular expressions [at the website Regular Expressions 101](https://regex101.com).

**Don't worry: you don't need to memorize all of this. The only thing you really need to know is how to use regex's to remove punctuation for our TTR task. But we thought you would enjoy seeing a demonstration of regex's immense power!**

## Removing Punctuation with Regular Expressions

Our adventures at Regular Expressions 101 will have showed us how we can quite simply remove "punctuation," which we will define as: 
- **any character that isn't a lowercase letter a-z, an uppercase letter A-Z, or a number 0-9.**

In the language of Regex, you would express that same definition as follows: 
- **[^a-zA-Z0-9]**

We want to grab all of those and replace them with **spaces** since replacing the `-` in a word like `seven-per-cent` would turn it into a non-word like `sevenpercent` rather than three separate words, `seven per cent`, which our `string.split()` method will be able to easily "tokenize."

In [ ]:
cocaine_exchange = sot4[1475:1843]
print(cocaine_exchange)

In [ ]:
cocaine_exchange = re.sub("[^a-zA-Z0-9]", " ", cocaine_exchange)
print(cocaine_exchange)

**That does the trick!** 

So now the question becomes: **WHEN** should we remove punctuation? When `sot4` is a string, or after we've used `.split()` to split it into words?

See the lecture notes where we try it both ways. 

In [ ]:
 # First we remove punctuation
cocaine_exchange_nopunct = re.sub("[^a-zA-Z0-9]", " ", cocaine_exchange)

# Then we split the text into words
cenp_words = cocaine_exchange_nopunct.split() 

cenp_unique_words = []

# By the time we enter this for loop, the punctuation is already gone
for word in cenp_words: 
    word = word.lower()
    if word not in cenp_unique_words:
        cenp_unique_words.append(word)

In [ ]:
cenp_unique_words[:10]

In [ ]:
(len(cenp_unique_words) / len(cenp_words)) * 100

### Now, let's do this for real

We are going to use a different text, because we want to compare TTRs by the end of the class. We have a folder collecting all the Presidential Inaugaral Addresses by U.S. presidents, from George Washington in 1789 to Joe Biden in 2021 (as they are collected [here](https://archive.org/details/Inaugural-Address-Corpus-1789-2009), supplemented from recent ones [here](https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/inaugural-addresses)).

We'll start with Washington's 1789 address, using Regex to remove all punctuation.

In [ ]:
# New method that removes punctuation
speech = open('speeches/1789-Washington.txt', encoding='utf-8').read()

# The variable names here, "np" signals "no punctuation"
speech_np = re.sub("[^a-zA-Z0-9]", " ", speech) 

speech_np_words = speech_np.split()

speech_np_unique_words = []

for word in speech_np_words:
    word = word.lower()
    if word not in speech_np_unique_words:
        speech_np_unique_words.append(word)

In [ ]:
speech[:500]

In [ ]:
speech_np[:500]

In [ ]:
speech_np_unique_words.sort()
speech_np_unique_words[:20]


The tokenization without punctuation works a lot better. 

### How much do you think this will affect the TTR?

In [ ]:
# Without removing the punctuation the TTR is 44.23

# When we remove the punctuation..
(len(speech_np_unique_words) / len(speech_np_words)) * 100

# 2. Iterating through Files in a Folder

Okay, we now know how to remove punctuation.

We need to be able to do three more things:
* Automatically create a standardized sample size
* Automate the loading of files so that we can can give Python a folder full of text files and let it do its thing, with no additional help from us
* Have Python spit out some nice spreadsheet files for us: one with non-standardized values and one with standardized values.

First, let's handle the folder-loading task.

## The `Path` function

We need another Python Library: `pathlib`, from which we are going to extract the function `Path`. We can coax it down the fireman's pole with this command:

In [ ]:
from pathlib import Path

`Path` will help us look through a folder and find the... **pathways** that Python needs to find files we want it to look and calculate TTR values for. 

First, `Path` needs to know the name of the folder that contains the files. Check for the folder `speeches` in Jupyter Lab.

In [ ]:
from pathlib import Path

# This variable can be named anything as long as is matches the variable name in the Path() command below
folder_path = "speeches"

file_list = Path(folder_path).glob("*")


The below command asks `Path` to look in a folder called `speeches` (which it expects to find **in the same folder as the Jupyter Notebook we are currently using — which it is!**), and to print out the paths of **absolutely everything in that folder** (the `"*"` as the argument to the `.glob()` method is what instructs it to look for everything).

In [ ]:
for file_path in Path(folder_path).glob("*"):
    print(file_path)

In [ ]:
for file_path in Path(folder_path).glob('*.txt'):
    print(file_path)

As you'll notice, the above list of files is not sorted, which is both aesthetically annoying and will also make it more difficult to interpret our results. (We can always sort things later using Microsoft Excel, etc., but we may as well do as much as we can right here in Python.)

Thankfully, we can wrap the Python function `sorted()` around the `Path()` function, and it will open the files in an alphabetically sorted manner.

In [ ]:
for file_path in sorted(Path(folder_path).glob('*.txt')):
    print(file_path)

Now, rather than just spewing out the file paths with the `print()` function... Let's actually **load** each of these files, and print out the first 100 characters of each, shall we?

In [ ]:
for file_path in sorted(Path(folder_path).glob('*.txt')):
    text = open(file_path, encoding='utf-8').read()
    print(text[:100])

Now that we can do this, we can take a major step: we can load an entire folder of files and, for each one, calculate its overall TTR. The code below does just that!

In [ ]:
import re
from pathlib import Path

folder_path = "speeches"

for file_path in sorted(Path(folder_path).glob('*.txt')):
    
    text = open(file_path, encoding='utf-8').read()
    text = re.sub("[^a-zA-Z0-9]", " ", text)
    
    text_words = text.split()
    tokens = len(text_words)
    
    unique_words = []
    
    for word in text_words:
        word = word.lower()
        if word not in unique_words:
            unique_words.append(word)
            
    types = len(unique_words)
    
    ttr = (types / tokens) * 100
    
    print(f"'{file_path.stem}' has {types} types, {tokens} tokens, and a TTR of {ttr:.2f}")

# 3. Automatically Determining Sample Size and Producing Standardized Results

To be specific, we still need to figure out
* How to automatically determine our sample size (i.e., the total length of the shortest text)
* And then how to calculate TTRs for that sample.

## Calculating the Total Length of the Shortest Item in a List

Before we do this with full-length texts, let's try with a small-scale experiment.

Let's say create a list with a bunch of strings in it. How could we automatically determine the total length of the shortest of these strings?

In [ ]:
bunch_o_strings = ["Marta", "Rosie", "Jazz", "Adamillo", "Anna", "Stephen", "Richard", "Ernest"]

In [ ]:
# Explain the code with your colleagues around you

length_of_shortest_string = 0 

for string in bunch_o_strings:
    string_length= len(string)
    if length_of_shortest_string == 0 or string_length < length_of_shortest_string:
        length_of_shortest_string = string_length

In [ ]:
length_of_shortest_string

## Creating Standardized Sample Sizes

Let's stick with our toy example for a moment.

We know that the `sample_size` we want is 4. So how to we actually *slice off* just the first 4 characters of each of those strings?

I'll show you the code, you tell me what the comment line does...

In [ ]:
for string in bunch_o_strings:
    string_standardized = string[:length_of_shortest_string] # What's happening here?
    print(string_standardized)

## We now have ALMOST all the skills we need to quickly and accurately calculate TTRs for any number of plain text files stored together in a folder

We already know how to:
* Automatically load files in a folder, one-by-one, as strings
* Remove punctuation, split them into words, and calculate their total number of words (tokens)
* Record the number of unique words (types) with a for loop that also lowercases all words
* Automatically standardize the sample size, looking only at the first x words in each text, where x is the total length of the shortest text

The only things we still don't know how to do are:
* Output the results of our analysis of the total texts (non-standardized results)
* Output the standardized results

Below I've written out the code that does all the things we've learned so far. It doesn't store the results anywhere yet, but it does calculate them and print them out.

This first cell records information for overall, non-standardized values and also determines our sample size (the total length of the shortest text). 

In [ ]:
import re
from pathlib import Path

folder_path = "speeches"

sample_size = 0  # Note this line and figure out what it's doing!

for file_path in sorted(Path(folder_path).glob('*.txt')):
    
    text = open(file_path, encoding='utf-8').read()
    text = re.sub("[^a-zA-Z0-9]", " ", text)
    
    text_words = text.split()
    tokens = len(text_words)
    
    if sample_size == 0 or tokens < sample_size: # The line I noted above connects to this one and the next
        sample_size = tokens
    
    unique_words = []
    
    for word in text_words:
        word = word.lower()
        if word not in unique_words:
            unique_words.append(word)
            
    types = len(unique_words)
    
    ttr = (types / tokens) * 100
    
    print(f"'{file_path.stem}' has {types} types, {tokens} tokens, and a TTR of {ttr}")
    print(f"So far, the shortest text is {sample_size} words in length.\n")

The cell below uses the sample size determined in the previous step (1769 words; stored in the `sample_size` variable) to calculate the standardized TTR for the first 1769 words of each text.

In [ ]:
for file_path in sorted(Path(folder_path).glob('*.txt')):
    text = open(file_path, encoding='utf-8').read()
    text = re.sub("[^a-zA-Z0-9]", " ", text)
    
    text_words = text.split()
    text_words_standardized = text_words[:sample_size] # This is the key new line in this block of code. Figure out what it does! Where did we define the variable sample_size?
    tokens_standardized = len(text_words_standardized)

    unique_words_standardized = []
    
    for word in text_words_standardized:
        word = word.lower()
        if word not in unique_words_standardized:
            unique_words_standardized.append(word)
            
    types_standardized = len(unique_words_standardized)
    
    ttr_standardized = (types_standardized / tokens_standardized) * 100
    
    print(f"'{file_path.stem}' has {types_standardized} types in the standardized sample of {tokens_standardized} tokens, and a TTR of {ttr_standardized:.2f}.\n")

# 4. Writing CSV files

Now let's complete the task by storing the results in a file.

We'll do this with the `open()` function, the `open.write()` method, and the CSV file format.

A CSV is a very simple file format for spreadsheets. 

A pretty table like the following:

| Types | Tokens                                                                                  |
|:-------------:|:---------------------------------------------------------------------------------------------------:|
| 12         | 24                                                                             |
| 33         | 100                                              |
| 75 | 500

Would be expressed in a CSV file as:

```
"Types","Tokens"
12,24
33,100
75,500
```

... and a spreadhseet program like Excel would see that and know exactly what to do with it.

(By the way, when Quercus gives your final grades at the end of the year to be uploaded to the eMarks system... it will be in the form of a CSV file!)

## Writing files with `open.write()`

We already know how to **load** a file into Python and stick it into a variable. We do it like this, with the `open()` function and the `.read()` method.

In [ ]:
sot4_file = open("sign-of-four.txt", encoding="utf-8")
sot4 = sot4_file.read()

Creating and writing to a file in Python happens almost the same wasy:
* First, you **`open()`** a file. Because it's not a file that already exists — it's one you're creating out of nothing — you need to set the *argument* `mode` to `"w"` (write) rather than the default `"r"` (read).
* Then you use the `.write()` method to write something into that file.


1. First, we `open()` a file and assign that "file object" to a variable called `file`. Because we want to write to it you need to set the *argument* `mode` to `"w"` (write) rather than the default `"r"` (read).
2. We then write whatever we want into that `file` varable, applying the `.write()` method. We can do this as many times as we like.
3. Then we "close" our file using the `.close()` method.  This final step is necessary to ensure that the data actually gets stored in the file. It tells the system that we are finished working with this file.

In [ ]:
file = open("a-new-file.txt", mode="w", encoding="utf-8")

file.write("Here's some text\n")
file.write("Here's some more!\n")
file.write("And that's all I want to write for now!")

file.close()

In [ ]:
print(open("a-new-file.txt", mode="r", encoding="utf-8").read())

Thankfully, this all works exactly the same way for a CSV file. Except... we put in commas between values and wrap text in `""`s

In [ ]:
file = open("my-first-spreadsheet.csv", mode="w", encoding="utf-8")

file.write('"Types","Tokens"\n') # Note that if you want to write double quotes, you have to wrap them in single quotes, or Python will get confused.
file.write("12,24\n")
file.write("33,100\n")
file.write("75,500")

file.close()

Now that we know how to write CSV files, we are well and truly finished with the TTR task (well... as much as anything is every finished! I can imagine a few ways to improve it, still. Can you??)

Here is the whole process, in a single cell. 

In [ ]:
import re
from pathlib import Path

folder_path = "speeches"

sample_size = 0

file = open("ttr-overall.csv", mode="w", encoding="utf-8")

file.write('"Text","Types","Tokens","TTR"\n')

for file_path in sorted(Path(folder_path).glob('*.txt')):
    print(file_path)
    
    text = open(file_path, encoding='utf-8').read()
    text = re.sub("[^a-zA-Z0-9]", " ", text)
    
    text_words = text.split()
    tokens = len(text_words)
    
    if sample_size == 0 or tokens < sample_size:
        sample_size = tokens
    
    unique_words = []
    
    for word in text_words:
        word = word.lower()
        if word not in unique_words:
            unique_words.append(word)
            
    types = len(unique_words)
    
    ttr = (types / tokens) * 100
    
    file.write(f'"{file_path.stem}",{types},{tokens},{ttr}\n')

file.close()



file = open("ttr-standardized.csv", mode="w", encoding="utf-8")

file.write('"Text","Types","Tokens","TTR"\n')

for file_path in sorted(Path(folder_path).glob('*.txt')):
    text = open(file_path, encoding='utf-8').read()
    text = re.sub("[^a-zA-Z0-9]", " ", text)
    
    text_words = text.split()
    text_words_standardized = text_words[:sample_size]
    tokens_standardized = len(text_words_standardized)

    unique_words_standardized = []
    
    for word in text_words_standardized:
        word = word.lower()
        if word not in unique_words_standardized:
            unique_words_standardized.append(word)
            
    types_standardized = len(unique_words_standardized)
    
    ttr_standardized = (types_standardized / tokens_standardized) * 100
    
    file.write(f'"{file_path.stem}",{types_standardized},{tokens_standardized},{ttr_standardized}\n')

file.close()

In [ ]:
''' One final step.  I asked ChatGPT to make me a graph of this data so that we could talk about it. After some back and forth to refine the output, here is the
code it gave me.  We will talk about many parts of this code in the remaining weeks of this course.
'''

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

# Load the data again
df = pd.read_csv("ttr-standardized.csv")

# Extract president names from the Text column
df["President"] = df["Text"].str.split("-", n=1).str[1]

# Create distinct colors for each president
unique_presidents = df["President"].unique()
cmap = cm.tab20  # any categorical colormap works (tab10, Set3, Paired, etc.)
colors = cmap(np.linspace(0, 1, len(unique_presidents)))

# Map each president to a color
color_map = {pres: colors[i] for i, pres in enumerate(unique_presidents)}
bar_colors = df["President"].map(color_map)

plt.figure(figsize=(14,7))

# Bar plot
x = np.arange(len(df))
plt.bar(x, df["TTR"], color=bar_colors)

# Add a linear trend line
z = np.polyfit(x, df["TTR"], 1)   # degree 1 polynomial (line)
p = np.poly1d(z)
plt.plot(x, p(x), "k--", linewidth=2, label="Trend line")

# X-axis setup
plt.xticks(ticks=x, labels=df["Text"], rotation=90, fontsize=8)
plt.xlabel("Speech")
plt.ylabel("TTR (Type-Token Ratio)")
plt.title("Type-Token Ratios of Speeches (Bar Graph, Color-coded by President + Trend Line)")

# Add legend
handles = [plt.Rectangle((0,0),1,1, color=color_map[pres]) for pres in unique_presidents]
plt.legend(handles + [plt.Line2D([0], [0], color="k", linestyle="--")],
           list(unique_presidents) + ["Trend line"],
           bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=8)

plt.tight_layout()
plt.show()